In [61]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np

In [62]:
reviews = pd.read_csv('C:/JK/Masters Studies/Spring 2018/Information Retrieval/Project/data/reviews.csv/reviews.csv')
# reviews['listing_id'].value_counts()
# vc = reviews['listing_id'].value_counts()

In [63]:
def remove_nonalphanumeric(text):
    return re.sub("[^a-zA-Z0-9]"," ", str(text)).lower()

In [64]:
unique_listings = reviews.listing_id.unique()
len(unique_listings)

3986

In [65]:
reviews=reviews.groupby('listing_id').filter(lambda g: (g.listing_id.size >= 4))
reviews = reviews[reviews['listing_id'] != 15041925]
reviews = reviews[reviews['listing_id'] !=19102778]

unique_listings = reviews.listing_id.unique()
len(unique_listings)

3005

In [66]:
reviews['comments_1'] = reviews['comments'].apply(remove_nonalphanumeric)
reviews['tokenized_comments'] = reviews['comments_1'].apply(word_tokenize)
stopset = stopwords.words('english') + list(string.punctuation)
reviews['stop_comments'] = reviews['tokenized_comments'].apply(lambda x: [item for item in x if item not in stopset])
def func(row):
    return " ".join(row)
reviews['data'] = reviews['stop_comments'].apply(lambda x: func(x))

In [67]:
def display_topics(H,W,feature_names,documents,documents_id,listing_id,words,no_top_words,no_top_documents):
    result_list = []
    for topic_idx, topic in enumerate(H):
        message = " ".join(([feature_names[i]
                             for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            result = (message,documents_id[doc_index],listing_id[doc_index],documents[doc_index],words[doc_index])
            result_list.append(result)

    return result_list

In [68]:
no_top_words = 5
no_top_documents = 10
result=[]

In [69]:
for each_listing in unique_listings:
    print(each_listing)
    
    listing_reviews=reviews[reviews['listing_id']==each_listing]
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=500)
    tf = tf_vectorizer.fit_transform(listing_reviews['data'])
    tf_feature_names = tf_vectorizer.get_feature_names()
    lda = LatentDirichletAllocation(n_topics = 5, max_iter = 5,learning_method='online',learning_offset=50., random_state=0).fit(tf)
    lda_W = lda.transform(tf)
    lda_H = lda.components_
    output = display_topics(lda_H,lda_W,tf_feature_names,list(listing_reviews['comments']),list(listing_reviews['id']),list(listing_reviews['listing_id']),list(listing_reviews['data']),no_top_words,no_top_documents)
    result += output

result_df = pd.DataFrame(result, columns=['Topic','Review_Id','Listing_Id','Documents','Words'])
result_df.to_csv("Result_entire.csv",columns=["Topic","Review_Id","Listing_Id","Documents","Words"])

7441144
12233830
14586440
15444930
1596470
8211468
19824694
8082512
20726840
17526985
13707814
16322051
11202183
3294775
15410121
5455293
20885932
11687031
4460532
14756184
619142
6908525
15470948
18155266
6390821
3881993
4085362
20031265
7692933
15342774
18053113
11866651
1688705
5088560
17761020
1810397
17190525
14107067
3129309
8302844
14315662
15852298
9448243
4787213
15408519
13752754
15857974
14125957
3589869
4603588
2021483
9148781
14647962
16030858
5690640
9570618
4105704
16352510
20023858
190170
5215982
12092499
50032
14072256
17156164
20870095
8732121
15397125
19295329
12351909
19344027
45987
7071883
990668
19328794
1494726
15507108
13054720
16600533
19415743
639111
12210223
15390344
20560553
2395326
4662379
16377190
12979965
4287989
11787441
4658261
8814532
17282895
19997604
10103725
9870
19631414
8065282
13063236
3605045
20504207
19144667
18557609
11972989
9858828
20623206
17892769
3865336
16282250
8751275
11590000
18137221
17838605
14251275
19720777
13236820
5581574
435818

14755734
10469962
19296971
16943329
8200839
3967592
18617774
14573951
4178790
11563716
11653496
18584041
5659866
1332786
67774
7036112
9477550
11662212
1454258
13778731
9845039
7511838
1175376
13685577
19017513
17237070
10857914
6911945
14300751
1301822
19103076
6886946
17510784
3987926
11788394
16239455
197972
17921621
9383459
17574145
2108738
8552338
3601254
4977034
10749312
10705351
801722
2186358
19160270
17559846
11470818
17559493
13377097
9757412
14682588
14709217
8634599
18777049
6629066
4025194
19315157
395975
17940636
17936801
10034183
14698794
14759485
11637504
8515356
6426364
9841140
4294557
14404992
163941
17889985
17989108
1815326
18175604
14442491
20635474
3528826
2187766
20554245
20817599
19787044
10231714
381187
20901417
4820071
3641545
18228183
1374434
8283433
6446940
16999403
20285405
19553261
12489916
8813451
15313517
19140676
5832127
21039321
21891
311240
15167383
20053362
11701611
798957
4358790
5478135
3966772
4670285
19744120
18016452
8067585
14668276
15797699
13

16453322
4526384
17006819
13056821
17440484
17936963
6214710
7731797
1321422
4132983
11431331
2526773
10138792
7805515
9057435
20666793
6677640
4107959
16413043
12103501
16259005
11240774
57156
4922204
15251516
19143984
19116365
13081090
12672097
13613229
19292171
8575476
15673767
9626428
447826
5959251
18841072
1881180
7797744
18755420
60356
16843072
6552654
13908031
12052288
7789429
2262752
18338493
12109171
4586681
13908638
18488956
17036313
13015767
14600925
4587068
5784404
3198169
11757166
508017
19102461
17702303
962610
18631999
18711
4567815
1885386
9301075
13907143
12233225
20164290
4860853
4555637
12492474
3591359
14065905
20589237
16286570
1327725
7511737
14522631
2292495
13171246
9414160
14663743
11648212
7755144
15139748
12372052
5593939
15139846
16276850
15581663
17399127
10117329
15885237
15407480
12191871
18440681
18254869
3079305
17494062
7482338
2378421
14041036
13078164
4623114
19540224
19143327
4547353
15038186
12233285
19180117
15314809
17359335
160572
5241043
44442

15423847
1810172
8962914
7474855
14083611
12962923
17710012
16904896
2701124
12143408
18377950
20929696
11589644
4687499
17669015
14402688
6954861
8482847
7479725
20094965
831744
14367642
14897219
17942707
13450030
17462718
15376003
20264910
11454147
18672885
8704828
20363107
18527992
15314772
15411577
6390634
11498583
15163837
4255538
20810636
1426711
16172058
7093109
19323221
5709298
4624582
2372987
8340072
20436407
9750460
12071820
4329915
2549840
19209297
15022444
18812080
5953696
13796564
10811
19676676
4173285
14415125
15390860
19927772
18672367
18181482
18473519
10427199
19823776
14428745
9259184
3903324
4863101
5782221
14887491
2515870
13673021
14124430
7665711
16002324
13466910
9494511
13017064
14646704
20175220
14566386
16540349
18209134
20711007
18823105
19384244
18575841
10004575
14994629
11710680
4699828
15453527
1340290
8780440
8227206
17856284
20763306
8388481
2592416
2033103
4593974
15140520
10945407
11970702
15851308
5870309
15773389
533114
